本文内容主要来自：
- baidu
- wiki
- zhihu
- http://zheming.wang/blog/2015/05/19/3AFF5BE8-593C-4F76-A72A-6A40FB140D4D/

# RPC

RPC主要是为了解决的两个问题：
- 解决分布式系统中，服务之间的调用问题
- 远程调用时，要能够像本地调用一样方便，让调用者感知不到远程调用的逻辑

RPC的实现过程
![欠一图]()
下面一步一步详细解释一下：
- Service A的应用层代码中，调用了Calculator的一个实现类的add方法，希望执行一个加法运算
- 这个Calculator实现类，内部并不是直接实现计算器的加减乘除逻辑，而是通过远程调用Service B的RPC接口，来获取运算结果，因此称之为Stub
- Stub怎么和Service B建立远程通讯呢？这时候就要用到远程通讯工具了，也就是图中的Run-time Library，这个工具将帮你实现远程通讯的功能，比如Java的Socket，就是这样一个库，当然，你也可以用基于Http协议的HttpClient，或者其他通讯工具类，都可以，RPC并没有规定说你要用何种协议进行通讯
- Stub通过调用通讯工具提供的方法，和Service B建立起了通讯，然后将请求数据发给Service B。需要注意的是，由于底层的网络通讯是基于二进制格式的，因此这里Stub传给通讯工具类的数据也必须是二进制，比如calculator.add(1,2)，你必须把参数值1和2放到一个Request对象里头（这个Request对象当然不只这些信息，还包括要调用哪个服务的哪个RPC接口等其他信息），然后序列化为二进制，再传给通讯工具类，这一点也将在下面的代码实现中体现
- 二进制的数据传到Service B这一边了，Service B当然也有自己的通讯工具，通过这个通讯工具接收二进制的请求
- 既然数据是二进制的，那么自然要进行反序列化了，将二进制的数据反序列化为请求对象，然后将这个请求对象交给Service B的Stub处理
- 和之前的Service A的Stub一样，这里的Stub也同样是个“假玩意”，它所负责的，只是去解析请求对象，知道调用方要调的是哪个RPC接口，传进来的参数又是什么，然后再把这些参数传给对应的RPC接口，也就是Calculator的实际实现类去执行。很明显，如果是Java，那这里肯定用到了反射
- RPC接口执行完毕，返回执行结果，现在轮到Service B要把数据发给Service A了，怎么发？一样的道理，一样的流程，只是现在Service B变成了Client，Service A变成了Server而已：Service B反序列化执行结果->传输给Service A->Service A反序列化执行结果 -> 将结果返回给Application，完毕

需要注意的是，虽然上面一直在讲的是客户和服务器不在一起的情况，但针对他们在同一台电脑的情况，也是可以使用的。

# DCOM
DCOM（分布式组件对象模型,分布式组件对象模式）是一系列微软的概念和程序接口，利用这个接口，客户端程序对象能够请求来自网络中另一台计算机上的服务器程序对象。DCOM基于组件对象模型（COM），COM提供了一套允许同一台计算机上的客户端和服务器之间进行通信的接口（运行在Windows95或者其后的版本上）<br>
在现在的操作系统中，各进程之间是相互屏蔽的。当一个客户进程需要和另一个进程中的组件通讯时，它不能直接调用该进程，而需要遵循操作系统对进程间通讯所做的规定。COM使得这种通讯能够以一种完全透明的方式进行：它截取从客户进程来的调用并将其传送到另一进程中的组件。图2表明了COM/DCOM运行库是怎样提供客户进程和组件之间的联系的<br>
当客户进程和组件位于不同的机器时，DCOM仅仅只是用网络协议来代替本地进程之间的通讯。无论是客户还是组件都不会知道连接它们的线路比以前长了许多。<br>

# CORBA
CORBA体系结构是对象管理组织（OMG）为解决分布式处理环境(DCE)中，硬件和软件系统的互连而提出的一种解决方案<br>
CORBA体系的主要内容包括以下几部分:
- 对象请求代理 （Object Request Broker，ORB）。负责对象在分布环境中透明地收发请求和响应，它是构建分布对象应用、在异构或同构环境下实现应用间互操作的基础
- 对象服务（Object Services）。为使用和实现对象而提供的基本对象集合，这些服务应独立于应用领域。主要的CORBA服务有：名录服务（Naming Service）、事件服务（Event Service）、生命周期服务（Life Cycle Service）、关系服务（Relationship Service）以及事务服务（Transaction Service）等。这些服务几乎包括分布系统和面向对象系统的各个方面，每个组成部分都非常复杂
- 公共设施（Common Facilitites）。向终端用户提供一组共享服务接口，例如系统管理、组合文档和电子邮件等
- 应用接口（Application Interfaces）。由销售商提供的可控制其接口的产品，相应于传统的应用层表示，处于参考模型的最高层。
- 领域接口（Domain Interfaces）。为应用领域服务而提供的接口，如OMG组织为PDM系统制定的规范

# Pyro
Pyro是一个用纯python代码实现的分布式对象系统，与早年的CORBA系统有很强的相似性，不过只支持Python的客户端和服务器端

# JavaSpace
JavaSpaces技术是进行分布式计算的一种简单机制，在分布式计算应用程序中，JavaSpaces 提供了对象的提供者和请求者可用来方便地进行通信的共享虚拟空间，这允许以Java对象的形式对任务、请求和信息进行简单的交换。Javaspaces提供了永久地建立和保存对象的能力，Javaspaces是建立在Jini之上的一种技术，它作为一种共享分布式通信的机制，还可作为一种存储对象的机制；JavaSpaces是一种Jini技术服务，这表示使用Javaspaces的开发人员也可以利用各种Jini服务比如说利用事务处理和通知等。JavaSpaces以一种非常简单的接口提供了这些功能，但是，有效地利用这种简单的接口需要大量的概念和工具。

# MapReduce
- 一个Map/Reduce 作业（job） 通常会把输入的数据（input file）切分为若干独立的数据块（splits），然后由 map任务（task）以完全并行的方式处理它们。Map/Reduce框架会对map的输出做一个 Shuffle 操作，Shuffle 操作的后的结果会输入给reduce任务。整个Map/Reduce框架负责任务的调度和监控，以及重新执行已经失败的任务。
- Map/Reduce计算集群由一个单独的JobTracker（master） 和每个集群节点一个 TaskTracker（slave）共同组成。JobTracker负责调度构成一个作业的所有任务，这些任务会被分派到不同的TaskTracker上去执行，JobTracker会监控它们的执行、重新执行已经失败的任务。而TaskTracker仅负责执行由JobTracker指派的任务。


下面按照顺序，依次介绍具体的工作过程

### 作业的提交和监控
JobClient是用户提交的作业与JobTracker交互的主要接口<br>
JobClient提交作业的过程如下：
- map/reduce程序通过runJob()方法新建一个JobClient实例
- 向JobTracker请求一个新jobID，通过JobTracker的getNewJobId()获取
-  检查作业输入输出说明。如果没有指定输出目录或者输出目录已经存在，作业将不会被提交，map/reduce程序； 输入作业划分split，如果划分无法计算（如：输入路径不存在），作业将不会被提交，错误返回给map/reduce程序
- 将运行作业所需要的资源（作业的jar文件、配置文件、计算所得的输入划分）复制到一个以作业ID命名的目录中；
- 通过调用JobTracker的submitJob()方法，告诉JobTracker作业准备提交
- JobTracker将提交的作业放到一个内部队列中，交由作业调度器进行调度，并对其进行初始化
- 创建Map任务、Reduce任务：一个split对应一个map，有多少split就有多少map; Reduce任务的数量由JobConf的mapred.reduce.tasks属性决定
- TaskTracker执行一个简单的循环，定期发送心跳（heartbeat）给JobTracker?????

### Input files
Input file是map/reduce任务的原始数据，一般存储在HDFS上。应用程序至少应该指明输入/输出的位置（路径），并通过实现合适的接口或抽象类提供map和reduce函数。再加上其他作业的参数，就构成了作业配置（job configuration）。然后，Hadoop的 job client提交作业（jar包/可执行程序等）和配置信息给JobTracker，后者负责分发这些软件和配置信息给slave、调度任务并监控它们的执行，同时提供状态和诊断信息给job-client。

### InputFormat
InputFormat为Map/Reduce作业输入的细节规范。Map/Reduce框架根据作业的InputFormat来：
- 检查作业输入的正确性，如格式等
- 把输入文件切分成多个逻辑InputSplit实例， 一个InputSplit将会被分配给一个独立的Map任务
- 提供RecordReader实现，这个RecordReader从逻辑InputSplit中获得输入记录（”K-V对”），这些记录将由Map任务处理

### InputSplits
InputSplit是一个单独的Map任务需要处理的数据块。一般的InputSplit是字节样式输入，然后由RecordReader处理并转化成记录样式。通常一个split就是一个block，这样做的好处是使得Map任务可以在存储有当前数据的节点上运行本地的任务，而不需要通过网络进行跨节点的任务调度。<br>

可以通过设置mapred.min.split.size， mapred.max.split.size, block.size来控制拆分的大小。如果mapred.min.split.size大于block size，则会将两个block合成到一个split，这样有部分block数据需要通过网络读取；如果mapred.max.split.size小于block size，则会将一个block拆成多个split，增加了Map任务数。<br>
在Map任务开始前，会先获取文件在HDFS上的路径和block信息，然后根据splitSize对文件进行切分（splitSize = computeSplitSize(blockSize, minSize, maxSize) ），默认splitSize 就等于blockSize的默认值（64m）<br>

### Mapper
Map是一类将输入记录集转换为中间格式记录集的独立任务，主要是读取InputSplit的每一个Key,Value对并进行处理

##### 确定Map任务的数量
Map/Reduce框架为每一个InputSplit产生一个map任务，而每个InputSplit是由该作业的InputFormat产生的,默认一个InputSplit大小就等于blockSize的默认值。因此，maps的数量通常取决于输入大小,也即输入文件的block数。 因此，假如输入数据有10TB，而block大小为64M，则需要164,000个map。map正常的并行规模大致是每个节点（node）大约10到100个map，对于CPU 消耗较小的map任务可以设到300个左右。<br>

因为启动任务也需要时间，所以在一个较大的作业中，最好每个map任务的执行时间不要少于1分钟，这样可以让启动任务的开销占比尽可能的低。对于那种有大量小文件输入的的作业来说，一个map处理多个文件会更有效率。如果输入的是打文件，那么一种提高效率的方式是增加block的大小（比如512M），每个map还是处理一个完整的HDFS的block。<br>

当在map处理的block比较大的时候，确保有足够的内存作为排序缓冲区是非常重要的，这可以加速map端的排序过程。假如大多数的map输出都能在排序缓冲区中处理的话应用的性能会有极大的提升。这需要运行map过程的JVM具有更大的堆。<br>

网格模式：确保map的大小，使得所有的map输出可以在排序缓冲区中通过一次排序来完成操作。<br>

合适的map数量有以下好处
- 减少了调度的负担；更少的map意味着任务调度更简单，集群中可用的空闲槽更多
- 有足够的内存将map输出容纳在排序缓存中，这使map端更有效率
- 减少了需要shuffle map输出的寻址次数，每个map产生的输出可用于每一个reduce，因此寻址数就是map个数乘以reduce个数
- 每个shuffle 的片段更大，这减少了建立连接的相对开销，所谓相对开销是指相对于在网络中传输数据的过程
- 这使reduce端合并map输出的过程更高效，因为合并的次数更少，因为需要合并的文件段更少了

#### 执行map任务
Mapper的实现者需要重写 JobConfigurable.configure(JobConf)方法，这个方法需要传递一个JobConf参数，目的是完成Mapper的初始化工作。然后，框架为这个任务的InputSplit中每个键值对调用一次 map(WritableComparable, Writable, OutputCollector, Reporter)操作。这里需要指出很多人的一种错误认识——“输入和输出的键值对类型一致，一一对应”，这种认识是错误的。输入输出键值对的关系如下：
- 输出键值对不需要与输入键值对类型一致
- 一个给定的输入键值对可以映射成0个或多个输出键值对

通过调用OutputCollector.collect(WritableComparable,Writable)可以收集map(WritableComparable, Writable, OutputCollector, Reporter)输出的键值对。应用程序可以使用Reporter报告进度，设定应用级别的状态消息，更新Counters（计数器），或者仅是表明自己运行正常。<br>
Map/Reduce框架随后会把与一个特定key关联的所有中间过程的值（value） 分组 并 排序 这个分组和排序过程被称为Shuffle,然后把它们传给Reducer以产出最终的结果。分组的总数目和一个作业的reduce任务的数目是一样的。用户可以通过实现自定义的 Partitioner来控制哪个key被分配给哪个 Reducer。 对于map的输出，用户可选择通过JobConf.setCombinerClass(Class)指定一个combiner，它负责对中间过程的输出进行本地的聚集，这会有助于降低从Mapper到 Reducer数据传输量。

### map输出溢写（spill） && Shuffle

#### shuffle
一般把从map任务输出到reducer任务输入之间的map/reduce框架所做的工作叫做shuffle。这部分也是map/reduce框架最重要的部分。
#### 内存缓冲区